In [1]:
import pandas as pd

In [2]:
data_census = pd.read_csv('Census_2010.csv')

In [3]:
data_census.head()

,Zipcode,Sex,Total,White,Black,Asian,Hispanic,Mixed Non-Hispanic
0,10001,Female,10918,"5,929",745,"1,987","1,913",265
1,10001,Male,10184,"5,775",857,"1,532","1,735",231
2,10002,Female,42840,"8,032","2,906","19,675","11,472",624
3,10002,Male,38570,"8,298","2,351","18,064","9,147",576
4,10003,Female,29274,"20,677",788,"4,620","2,223",825


In [8]:
data_311 = pd.read_csv('311_Service_Requests_from_2010_to_Present.csv')

In [9]:
data_311

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Incident Zip,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Latitude,Longitude,Location
0,19503049,12/31/2010 11:59:12 PM,01/01/2011 02:14:44 AM,NYPD,New York City Police Department,Blocked Driveway,11212,BROOKLYN,1008500,181726,40.665439,-73.912589,"(40.665439409429055, -73.91258883811614)"
1,19503621,12/31/2010 11:57:30 PM,01/01/2011 12:46:41 AM,NYPD,New York City Police Department,Blocked Driveway,11365,QUEENS,1037441,205363,40.730191,-73.808082,"(40.73019074217415, -73.80808214665396)"
2,19504838,12/31/2010 11:54:59 PM,01/01/2011 08:03:26 AM,NYPD,New York City Police Department,Blocked Driveway,11236,BROOKLYN,1006381,176021,40.649786,-73.920246,"(40.64978599344521, -73.9202456701406)"
3,19506107,12/31/2010 11:54:00 PM,03/16/2011 09:36:00 AM,DOT,Department of Transportation,Street Light Condition,10006,MANHATTAN,980807,197440,40.708603,-74.012419,"(40.70860327698897, -74.01241861690464)"
4,19507090,12/31/2010 11:52:00 PM,02/01/2011 12:45:00 PM,DEP,Department of Environmental Protection,Noise,11209,BROOKLYN,978741,169492,40.631891,-74.019848,"(40.63189122768689, -74.01984766770063)"
5,19504837,12/31/2010 11:50:45 PM,01/01/2011 02:10:59 AM,NYPD,New York City Police Department,Blocked Driveway,11368,QUEENS,1021079,214444,40.755199,-73.867068,"(40.755199052630154, -73.86706786330053)"
6,19502935,12/31/2010 11:48:11 PM,01/01/2011 03:39:55 AM,NYPD,New York City Police Department,Illegal Parking,10003,MANHATTAN,987254,202897,40.723582,-73.989162,"(40.723581595109145, -73.98916238466072)"
7,19506202,12/31/2010 11:48:00 PM,01/01/2011 01:35:00 AM,DOT,Department of Transportation,Traffic Signal Condition,11228,BROOKLYN,981784,162247,40.612007,-74.008882,"(40.61200658490669, -74.00888178783936)"
8,19506287,12/31/2010 11:48:00 PM,12/31/2010 11:48:00 PM,DSNY,BCC - Queens East,Snow,11422,QUEENS,1059628,181045,40.663283,-73.728302,"(40.66328256317842, -73.72830206247659)"
9,19507089,12/31/2010 11:44:00 PM,01/03/2011 03:36:00 PM,DEP,Department of Environmental Protection,Noise,11235,BROOKLYN,995108,151042,40.581245,-73.960911,"(40.581244814092614, -73.96091077190891)"
